In [1]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

In [2]:
from scispacy.data_util import read_full_med_mentions, read_ner_from_tsv
from scispacy.per_class_scorer import PerClassScorer
from scispacy.train_utils import evaluate_ner

In [3]:
def train_ner(output_dir: str,
              train_data_path: str,
              dev_data_path: str,
              test_data_path: str,
              run_test: bool = None,
              model: str = None,
              n_iter: int = 10,
              meta_overrides: str = None):

    util.fix_random_seed(util.env_opt("seed", 0))
    train_data = read_ner_from_tsv(train_data_path)
    dev_data = read_ner_from_tsv(dev_data_path)
    test_data = read_ner_from_tsv(test_data_path)
    os.makedirs(output_dir, exist_ok=True)
    if run_test:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
        evaluate_ner(nlp, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
        evaluate_ner(nlp, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))
    else:
        train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides)

In [4]:
path_to_pretrained_weights = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/gold_pretrained/model41.bin'
# otherwise, get it so we can add labels
def train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides):
    """Load the model, set up the pipeline and train the entity recognizer."""
    
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    if meta_overrides is not None:
        metadata = json.load(open(meta_overrides))
        nlp.meta.update(metadata)

    original_tokenizer = nlp.tokenizer

#     nlp.tokenizer = nlp_en.tokenizer

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names and "parser" in nlp.pipe_names:
        ner = nlp.create_pipe("ner", config={"architecture": "simple_cnn", "exclusive_classes": True})
        nlp.add_pipe(ner, after="parser")
    elif 'ner' not in nlp.pipe_names and "tagger" in nlp.pipe_names:
        ner = nlp.create_pipe("ner", config={"architecture": "simple_cnn", "exclusive_classes": True})
        nlp.add_pipe(ner, after="tagger")
    elif 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe("ner", config={"architecture": "simple_cnn", "exclusive_classes": True})
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    dropout_rates = util.decaying(util.env_opt('dropout_from', 0.2),
                                  util.env_opt('dropout_to', 0.2),
                                  util.env_opt('dropout_decay', 0.005))
    batch_sizes = util.compounding(util.env_opt('batch_from', 1),
                                   util.env_opt('batch_to', 64),
                                   util.env_opt('batch_compound', 1.001))

    optimizer = nlp.begin_training()
    
    
# Now that we have our model, we can load in the pretrained weights.
    with open(path_to_pretrained_weights, "rb") as file_:
        ner.model.tok2vec.from_bytes(file_.read())
    
    best_epoch = 0
    best_f1 = 0
    for i in range(n_iter):
        print(str(i)+'--'+str(n_iter))
        random.shuffle(train_data)
        count = 0
        losses = {}
        total = len(train_data)

        with nlp.disable_pipes(*other_pipes):  # only train NER
            with tqdm.tqdm(total=total, leave=True) as pbar:
                for batch in minibatch(train_data, size=batch_sizes):
                    docs, golds = zip(*batch)
                    nlp.update(docs, golds, sgd=optimizer,
                               losses=losses, drop=next(dropout_rates))
                    pbar.update(len(batch))
                    if count % 100 == 0 and count > 0:
                        print('sum loss: %s' % losses['ner'])
                    count += 1

        # save model to output directory
        output_dir_path = Path(output_dir + "/" + str(i))
        if not output_dir_path.exists():
            output_dir_path.mkdir()

        with nlp.use_params(optimizer.averages):
            nlp.tokenizer = original_tokenizer
            nlp.to_disk(output_dir_path)
            print("Saved model to", output_dir_path)

        # test the saved model
        print("Loading from", output_dir_path)
        nlp2 = util.load_model_from_path(output_dir_path)
#         nlp2.tokenizer = nlp_en.tokenizer

        metrics = evaluate_ner(nlp2, dev_data)
        if metrics["f1-measure-overall"] > best_f1:
            best_f1 = metrics["f1-measure-overall"]
            best_epoch = i
    # save model to output directory
    best_model_path = Path(output_dir + "/" + "best")
    print(f"Best Epoch: {best_epoch} of {n_iter}")
    if os.path.exists(best_model_path):
        shutil.rmtree(best_model_path)
    shutil.copytree(os.path.join(output_dir, str(best_epoch)),
                    best_model_path)

    # test the saved model
    print("Loading from", best_model_path)
    nlp2 = util.load_model_from_path(best_model_path)
#     nlp2.tokenizer = nlp_en.tokenizer

    evaluate_ner(nlp2, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
    evaluate_ner(nlp2, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))

In [5]:
model_output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pretrain_exp/'
train_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/train.csv'
dev_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/dev.csv'
test_data_path = '/nfs/gns/literature/machine-learning/Datasets/NER_Datasets/EBI_standard-IOB/test.csv'
run_test = False
model_path = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/pubmed-pmc/' # None #'en_core_sci_md'
iterations = 7
meta_overrides = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/scispacy/data/EPMC_ner.json'

In [ ]:
train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations
             )

Loaded model '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/pubmed-pmc/'


  0%|          | 4/79401 [00:00<41:02, 32.25it/s]

0--7


  0%|          | 104/79401 [00:03<39:47, 33.21it/s]

sum loss: 211.39885538931108


...

 81%|████████  | 64128/79401 [02:28<00:36, 424.10it/s]

sum loss: 8213.667462518748


 89%|████████▉ | 70528/79401 [02:43<00:18, 482.33it/s]

sum loss: 9139.028734635114


 93%|█████████▎| 73728/79401 [02:50<00:12, 443.43it/s]